In [1]:
import pandas as pd
import cornac
from recommenders.datasets.python_splitters import python_random_split
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.constants import SEED
from recommenders.utils.timer import Timer
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
import torch

In [2]:
# datacsv = pd.read_csv("data/user_recipe.csv")
# datacsv.columns = ["userID", "itemID", "rating", "modifyDate"]
datacsv = pd.read_csv("data/new_user_recipe.csv")
datacsv.columns = ["idx", "itemID", "userID", "text", "rating"]
datacsv.drop(["idx"], inplace=True, axis=1)
datacsv = datacsv[["userID", "itemID", "rating", "text"]]

In [3]:
train, test = python_random_split(datacsv, 0.75)
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 480
Number of items: 294


/home/vscode/.local/lib/python3.9/site-packages/cornac/data/dataset.py:361: UserWarning: 20417 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [4]:
# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 128
LEARNING_RATE = 0.001

bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 66.2484 seconds for training.


In [9]:
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 0.2548 seconds for prediction.


In [10]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.028668
NDCG:	0.211674
Precision@K:	0.204167
Recall@K:	0.058376
